In [ ]:
#1 imports
import torch
import torch.nn as nn
import torch.nn.functional as F #nonlinearilization ReLU
import torch.optim as optim #Adam, adagard etc

from torch.utils.data import  DataLoader #minibatches, shuffling, sgd etc
from torch.utils.data import Dataset

import torchvision.datasets as datasets #dataset mnist
import torchvision.transforms as transforms #convertion to tensor

from torchvision.transforms import ToTensor, ToPILImage
from torchvision.transforms.functional import rotate

In [ ]:
class CustomRotatedMNIST(Dataset):
  def __init__(self, train=True):
    self.base_dataset = datasets.MNIST(root='./data', train=train, download=True, transform=transforms.ToTensor())
    self.rotation_angles = [0, 90, 180, 270]
    self.rotation_labels = {0:0, 90:1, 180:2, 270:3}
    self.valid_digits = [1, 2, 3, 4, 5, 7]  # sadece bu rakamlar kullanılacak
    self.label_map = {label: idx for idx, label in enumerate(self.valid_digits)}

    self.samples = []
    to_pil = ToPILImage()
    to_tensor = ToTensor()

    for img_tensor, digit_label in self.base_dataset:
      if digit_label not in self.valid_digits:
          continue

      new_label = self.label_map[digit_label]  # yeniden numaralandır (örneğin 1 → 0, 2 → 1, ...)

      for angle in self.rotation_angles:
        rotated_img = rotate(to_pil(img_tensor),angle)
        rotated_tensor = to_tensor(rotated_img)
        self.samples.append((rotated_tensor, new_label, self.rotation_labels[angle]))

  def __len__(self):
      return len(self.samples)

  def __getitem__(self, idx):
      image, digit_label, rotation_label = self.samples[idx]
      return image, digit_label, rotation_label


In [ ]:
#2 creat FCNN
class NN (nn.Module):
    def __init__(self, input_size,num_classes): #28x28=784 input size, 10 classes
        super(NN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=8,kernel_size=(3,3), stride=(1,1),padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2),stride=(1,1),padding=(1,1))
        self.conv2 = nn.Conv2d(in_channels=8,out_channels=16,kernel_size=(3,3), stride=(1,1),padding=(1,1))
        self.fc1 = nn.Linear(14400, 128)
        self.fc_digit = nn.Linear(128, num_classes)
        self.fc_rotation = nn.Linear(128, 4)
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1) #flattening the data
        x = self.fc1(x)
        x = F.relu(x)
        digit_output = self.fc_digit(x)
        rotation_output = self.fc_rotation(x)
        return digit_output, rotation_output

In [ ]:
def train(model, train_loader, optimizer, criterion_digit, criterion_rotation, device, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch_idx, (data, digit_labels, rotation_labels) in enumerate(train_loader):
            data = data.to(device)
            digit_labels = digit_labels.to(device)
            rotation_labels = rotation_labels.to(device)

            digit_output, rotation_output = model(data)
            loss_digit = criterion_digit(digit_output, digit_labels)
            loss_rotation = criterion_rotation(rotation_output, rotation_labels)
            loss = loss_digit + loss_rotation

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{num_epochs} | Avg Loss: {avg_loss:.4f}")

In [ ]:
def evaluate(model, loader, device):
    model.eval()

    all_digit_preds = []
    all_digit_labels = []
    all_rot_preds = []
    all_rot_labels = []

    with torch.no_grad():
        for x, digit_labels, rotation_labels in loader:
            x = x.to(device)
            digit_labels = digit_labels.to(device)
            rotation_labels = rotation_labels.to(device)

            digit_out, rotation_out = model(x)
            digit_pred = digit_out.argmax(dim=1)
            rot_pred = rotation_out.argmax(dim=1)

            all_digit_preds.append(digit_pred.cpu())
            all_digit_labels.append(digit_labels.cpu())
            all_rot_preds.append(rot_pred.cpu())
            all_rot_labels.append(rotation_labels.cpu())

    # Hepsini tek tensöre çevir
    all_digit_preds = torch.cat(all_digit_preds)
    all_digit_labels = torch.cat(all_digit_labels)
    all_rot_preds = torch.cat(all_rot_preds)
    all_rot_labels = torch.cat(all_rot_labels)

    # Metrikleri hesapla
    digit_metrics = calculate_metrics(all_digit_preds, all_digit_labels, num_classes=6)
    rotation_metrics = calculate_metrics(all_rot_preds, all_rot_labels, num_classes=4)

    print("\n📊 Digit Classification Metrics:")
    print(f"Accuracy      : {digit_metrics['accuracy']*100:.2f}%")
    print(f"Macro F1      : {digit_metrics['macro_f1']:.4f}")
    print(f"Precision     : {digit_metrics['macro_precision']:.4f}")
    print(f"Recall        : {digit_metrics['macro_recall']:.4f}")

    print("\n📊 Rotation Classification Metrics:")
    print(f"Accuracy      : {rotation_metrics['accuracy']*100:.2f}%")
    print(f"Macro F1      : {rotation_metrics['macro_f1']:.4f}")
    print(f"Precision     : {rotation_metrics['macro_precision']:.4f}")
    print(f"Recall        : {rotation_metrics['macro_recall']:.4f}")

    model.train()

In [ ]:
def calculate_metrics(preds, targets, num_classes):
    TP = torch.zeros(num_classes)
    FP = torch.zeros(num_classes)
    FN = torch.zeros(num_classes)

    for cls in range(num_classes):
        TP[cls] = ((preds == cls) & (targets == cls)).sum().item()
        FP[cls] = ((preds == cls) & (targets != cls)).sum().item()
        FN[cls] = ((preds != cls) & (targets == cls)).sum().item()

    precision = TP / (TP + FP + 1e-8)
    recall = TP / (TP + FN + 1e-8)
    f1 = 2 * precision * recall / (precision + recall + 1e-8)
    accuracy = (preds == targets).sum().item() / len(targets)

    macro_precision = precision.mean().item()
    macro_recall = recall.mean().item()
    macro_f1 = f1.mean().item()

    print(f"TP: {TP}")
    print(f"FP: {FP}")
    print(f"FN: {FN}")

    return {
        "accuracy": accuracy,
        "macro_precision": macro_precision,
        "macro_recall": macro_recall,
        "macro_f1": macro_f1
    }


In [ ]:
def main():
    #4 set the hyperparameters
    input_size= 784 #28x28
    num_classes = 6 #0-9
    learning_rate = 0.001
    batch_size= 64
    num_epochs=10

    #3 set the device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Dataset ve DataLoader
    train_dataset = CustomRotatedMNIST(train=True)
    test_dataset = CustomRotatedMNIST(train=False)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Model
    model = NN(input_size=input_size, num_classes=num_classes)

    # Loss ve Optimizer
    criterion_digit = nn.CrossEntropyLoss()
    criterion_rotation = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Eğitim
    train(model, train_loader, optimizer, criterion_digit, criterion_rotation, device, num_epochs)

    # Test
    evaluate(model, test_loader, device)

    # Modeli Kaydet
    torch.save(model.state_dict(), "digit_rotation_model.pth")
    print("\n✅ Model saved as digit_rotation_model.pth")


# 🔁 Giriş noktası (özellikle .py dosyası olarak çalıştırırken önemlidir)
if __name__ == "__main__":
    main()


Epoch 1/10 | Avg Loss: 0.2460
Epoch 2/10 | Avg Loss: 0.0778
Epoch 3/10 | Avg Loss: 0.0574
Epoch 4/10 | Avg Loss: 0.0458
Epoch 5/10 | Avg Loss: 0.0368
Epoch 6/10 | Avg Loss: 0.0312
Epoch 7/10 | Avg Loss: 0.0264
Epoch 8/10 | Avg Loss: 0.0225
Epoch 9/10 | Avg Loss: 0.0190
Epoch 10/10 | Avg Loss: 0.0173
TP: tensor([4527., 4070., 3976., 3900., 3513., 4053.])
FP: tensor([51., 47., 21., 43., 41., 74.])
FN: tensor([13., 58., 64., 28., 55., 59.])
TP: tensor([6048., 6061., 6054., 6054.])
FP: tensor([18., 30., 25., 26.])
FN: tensor([31., 18., 25., 25.])

📊 Digit Classification Metrics:
Accuracy      : 98.86%
Macro F1      : 0.9885
Precision     : 0.9886
Recall        : 0.9884

📊 Rotation Classification Metrics:
Accuracy      : 99.59%
Macro F1      : 0.9959
Precision     : 0.9959
Recall        : 0.9959

✅ Model saved as digit_rotation_model.pth
